<div class="header">
  <img src="img/kg_logo_white_side.png" alt="logo" style="width: 300px;"/>
    <h1> Elliptic Dataset / GNN Embedding Pipeline </h1>
</div>

  <img src="img/pipeline.png" alt="logo" style="width: 1000px;"/>

In [1]:
import yaml
import warnings

with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
project_id = config['project']['project_id']

## PREPROCESSING & INGESTION

<img src="img/dask.png" alt="Drawing" style="width: 600px;"/>

In [2]:
from src.elliptic_dask_preprocessing import elliptic_preprocessing
elliptic_graph = elliptic_preprocessing()

Table deleted: katana-clusters-beta.fsi_elliptic.account_features
Loaded 203769 rows and 95 columns to katana-clusters-beta.fsi_elliptic.account_features
Updated elliptic_feast/feature_repo/elliptic_features.py with baseline feature view


          0/? [?op/s]

          0/? [?op/s]

In [3]:
%%time
elliptic_graph.query("MATCH (m)-->(n) RETURN * LIMIT 500", contextualize=True).visualize()

          0/? [?op/s]

          0/? [?op/s]

CPU times: user 3.57 s, sys: 301 ms, total: 3.87 s
Wall time: 39.9 s


GraphVizWidget(edges=[{'from': 43677, 'to': 44769, 'label': 'rev_transaction', 'type': 'rev_transaction', 'pro…

## ENCODING & TRAINING

<img src="img/gnn_train.png" alt="Drawing" style="width: 700px;"/>

In [4]:
from src.katana_enterprise import GNNEmbeddingPipeline
import sys,os
sys.path.append(os.path.join("~/solutions/fsi/demos/elliptic"))
elliptic_node_classification_job = GNNEmbeddingPipeline(
    display_name="elliptic_gnn_embedding_train_job", 
    optimization_prediction_type="node_classification",
    embed_dim=16,
    supervised=True
)

In [5]:
%%time
import warnings
elliptic_node_classification_job.train(
    elliptic_graph,
    target_property_name="target",
    graph_analytics_features=["page_rank", "betweenness_centrality"],
    budget_milli_node_hours=8000,
    model_display_name="elliptic_gnn_embed",
    disable_early_stopping=False,
    sync=True,
    split_ratio=[0.8, 0.15, 0.05],
)

          0/? [?op/s]

          0/? [?op/s]

          0/? [?op/s]


Host 0 output:
Tensorboard log directory: gs://katana-internal1/tensorboard/elliptic_demo_e2e_89e41fda2e214b63b664ec8ae6f2c2ba
Validation data test_score: {'validation_metric': 0.9681075566363395}
Test data test_score: {'validation_metric': 0.9590854646656802}

Host 1 output:
Tensorboard log directory: gs://katana-internal1/tensorboard/elliptic_demo_e2e_89e41fda2e214b63b664ec8ae6f2c2ba
Validation data test_score: {'validation_metric': 0.9561765038027762}
Test data test_score: {'validation_metric': 0.9437159727014799}

Host 2 output:
Tensorboard log directory: gs://katana-internal1/tensorboard/elliptic_demo_e2e_89e41fda2e214b63b664ec8ae6f2c2ba
Validation data test_score: {'validation_metric': 0.9650508216965897}
Test data test_score: {'validation_metric': 0.8946277169724065}

Host 3 output:
Tensorboard log directory: gs://katana-internal1/tensorboard/elliptic_demo_e2e_89e41fda2e214b63b664ec8ae6f2c2ba
Validation data test_score: {'validation_metric': 0.9535947908539879}
Test data test_s


Host 0 errors:

Host 1 errors:

Host 2 errors:

Host 3 errors:


## EMBEDDING  & STORAGE

<img src="img/gnn_inference.png" alt="Drawing" style="width: 700px;"/>

In [6]:
%%time
pipeline_uri = elliptic_node_classification_job.infer_embeddings(
    elliptic_graph, 
    project_id
    )

          0/? [?op/s]


Host 0 output:
Table created: katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 18 columns to katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba

Host 1 output:
Table created: katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 18 columns to katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba

Host 2 output:
Table created: katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 18 columns to katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba

Host 3 output:
Table created: katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 18 columns to katana-clusters-beta.fsi_elliptic.elliptic_embeddings_89e41fda2e214b63b664ec8ae6f2c2ba


          0/? [?op/s]


Host 0 output:
Table created: katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 6 columns to katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba

Host 1 output:
Table created: katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 6 columns to katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba

Host 2 output:
Table created: katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 6 columns to katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba

Host 3 output:
Table created: katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba
Loaded 203769 rows and 6 columns to katana-clusters-beta.fsi_elliptic.account_mapping_89e41fda2e214b63b664ec8ae6f2c2ba
CPU times: user 975 ms, sys: 45.6 ms, total: 1.02 s
Wall time: 25.5 

In [7]:
print(pipeline_uri)

89e41fda2e214b63b664ec8ae6f2c2ba


In [8]:
if (config['model']['pipeline_uri'] != pipeline_uri) or (config['model']['pipeline_uri'] is None):
    config['model']['pipeline_uri'] = pipeline_uri
with open('config.yaml', 'w') as f:
    config = yaml.dump(config, stream=f,
                       default_flow_style=False, sort_keys=False)

## GRAPH FEATURES & ACCURACY IMPROVEMENT

<img src="img/f1score.png" alt="Drawing" style="width: 1000px;"/>

### Graph Feature Importance

In [ ]:
%matplotlib inline

ax = xgboost.plot_importance(bst_embed, height=0.8, max_num_features=15)
ax.grid(False, axis="y")
ax.set_title('Estimated feature importance')
plt.show()

<div>
    <div style="margin-left: 50px;float:left">
        <b> Baseline </b>
<img src="img/baseline.png" alt="Drawing" style="width: 500px;"/>
    </div>
    <div style="margin-left: 50px;float:left">
        <b> With Graph Embeddings</b>
<img src="img/enhanced.png" alt="Drawing" style="width: 500px;"/>
    </div>
</div>